In [1]:
import json
import re
from pathlib import Path

# Caminhos
input_path = "../data/raw/iphone.ibyte.json"
output_path = "../data/annotations/iphone_auto_annotations.jsonl"

# ---------- Helpers ----------
def trim_span(text, start, end):
    """
    Remove espaços no início/fim do span e garante start < end.
    Retorna (start, end) ajustado.
    """
    # move start pra frente se começa em espaço
    while start < end and text[start].isspace():
        start += 1
    # move end pra trás se termina em espaço (end é exclusivo)
    while end > start and text[end - 1].isspace():
        end -= 1
    return start, end

def remove_overlaps(entities):
    """
    Remove sobreposições. Mantém a entidade mais longa quando houver conflito;
    se mesmo comprimento, mantém a que apareceu primeiro.
    Entities = list de [start, end, label]
    """
    sorted_e = sorted(entities, key=lambda x: (x[0], -(x[1]-x[0])))
    result = []
    for e in sorted_e:
        s, ee, lab = e
        conflict = False
        for i, (rs, re, rlab) in enumerate(result):
            # se sobrepõe
            if not (ee <= rs or s >= re):
                conflict = True
                # se nova é maior, substitui
                if (ee - s) > (re - rs):
                    result[i] = [s, ee, lab]
                # caso contrário mantém existente
                break
        if not conflict:
            result.append([s, ee, lab])
    # ordenar por start
    return sorted(result, key=lambda x: x[0])

def fix_touching_entities(entities, text):
    """
    Garante que não existam spans que comecem/terminem em espaços
    e evita spans encostados que incluam espaços.
    Ajusta para que, se uma entidade termina e a próxima começa exatamente
    no mesmo índice por conta de espaços, o espaço fique fora das entidades.
    """
    fixed = []
    for s, e, lab in entities:
        s, e = trim_span(text, s, e)
        if s >= e:
            continue
        if fixed:
            ps, pe, pl = fixed[-1]
            # Se a anterior termina exatamente onde a atual começa (ps..pe) == (s..e)
            # e entre elas há um espaço, garante que o espaço não esteja marcado.
            # Caso pe == s, não há sobreposição — já OK.
            # Se pe > s (sobreposição) isso já foi tratado em remove_overlaps.
            if pe == s and text[s:s+1].isspace():
                # move o início da atual para pular o espaço
                new_s = s + 1
                # trim again in case multiple spaces
                new_s, e = trim_span(text, new_s, e)
                # se ao pular espaço a entidade fica inválida, descartamos
                if new_s >= e:
                    continue
                s = new_s
        fixed.append([s, e, lab])
    return fixed

# ---------- Extração ----------
def extract_entities(text, brand=None):
    entities = []

    # 1) CATEGORIA -> detecta a palavra "iPhone" (case-insensitive)
    idx_tipo = re.search(r"\biphone\b", text, re.IGNORECASE)
    if idx_tipo:
        s, e = idx_tipo.start(), idx_tipo.end()
        s, e = trim_span(text, s, e)
        if s < e:
            entities.append([s, e, "CATEGORIA"])

    # 2) MARCA (se fornecida)
    if brand:
        m = re.search(re.escape(brand), text, re.IGNORECASE)
        if m:
            s, e = trim_span(text, m.start(), m.end())
            if s < e:
                entities.append([s, e, "MARCA"])

    # 3) MODELO -> somente o número + sufixo (se houver).
    # regex: número (1-2 dígitos) seguido opcionalmente por (space + Pro/Pro Max/Mini)
    modelo = re.search(r"\b(?:\d{1,2}(?:\s(?:Pro(?:\s?Max)?|Mini))?|SE)\b", text, re.IGNORECASE)
    if modelo:
        s, e = trim_span(text, modelo.start(), modelo.end())
        if s < e:
            entities.append([s, e, "MODELO"])

    # 4) MEMORIA (ex: 64GB, 128GB, 256GB, 512GB)
    memoria = re.search(r"\b\d{2,4}GB\b", text, re.IGNORECASE)
    if memoria:
        s, e = trim_span(text, memoria.start(), memoria.end())
        if s < e:
            entities.append([s, e, "MEMORIA"])

    # 5) COR (lista manual)
    cores = ["Preto", "Branco", "Vermelho", "Red", "Azul", "Roxo", "Verde",
            "Dourado", "Prateado", "Gold", "Midnight", "Starlight", "Pink", "Purple"]
    for cor in cores:
        m = re.search(r"\b" + re.escape(cor) + r"\b", text, re.IGNORECASE)
        if m:
            s, e = trim_span(text, m.start(), m.end())
            if s < e:
                entities.append([s, e, "COR"])

    # 6) REFERENCIA (mais restrita: letras+digitos com / e letra no final, ex: MLQ93BZ/A)
    ref = re.search(r"\b[A-Z0-9]{4,}[A-Z0-9]*(?:\/[A-Z0-9]+)?\b", text)
    if ref:
        # further filter: ensure it contains at least one letter or pattern length > 5
        s0, e0 = ref.start(), ref.end()
        span = text[s0:e0]
        if re.search(r"[A-Z]", span, re.IGNORECASE) and len(span) >= 5:
            s, e = trim_span(text, s0, e0)
            if s < e:
                entities.append([s, e, "REFERENCIA"])

    # 7) Remove overlaps e ajusta spans encostados
    entities = remove_overlaps(entities)
    entities = fix_touching_entities(entities, text)
    # ordenar
    entities.sort(key=lambda x: x[0])
    return entities

# ---------- Processamento do arquivo ----------
def generate_annotations(input_path, output_path):
    input_path = Path(input_path)
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(output_path, "w", encoding="utf-8") as out:
        for item in data:
            product = item["microdata"][1]
            name = product["name"]
            brand = product.get("brand", {}).get("name", None)

            entities = extract_entities(name, brand)
            record = {"text": name, "entities": entities}
            out.write(json.dumps(record, ensure_ascii=False) + "\n")

    print(f" Arquivo JSONL gerado em: {output_path}")

# Rodar
if __name__ == "__main__":
    generate_annotations(input_path, output_path)


 Arquivo JSONL gerado em: ../data/annotations/iphone_auto_annotations.jsonl


In [2]:
import os
os.getcwd()


'/workspaces/mt2025-2-ner-ecommerce-ner_e-commerce/notebooks'